In [1]:
import pandas as pd
import numpy as np
import pyodbc
from scipy.stats import shapiro
from scipy.stats import kstest
from configparser import ConfigParser
from scipy import stats as s
from datetime import datetime
# import import_ipynb
# import Entity_Risk

In [2]:
pd.set_option("display.max_column",200)
pd.set_option("display.float_format",lambda x:'%.5f' %x)

In [3]:
data_ind_U = pd.read_csv(r"/data/peer_updated.csv")

In [5]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [9]:
risk_lkp = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] with (nolock) ", conn)
auto_high_rules =  risk_lkp[(risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['RISK_TYPE'] == 'Auto High Rules ')]['RULE_NO'].to_list()
static_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 1) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Organization') ]['RULE_NO'].to_list()
non_stat_rules = risk_lkp[(risk_lkp['RISK_SCORE_STATIC_FLAG'] == 0) & (risk_lkp['IS_ACTIVE_FLAG'] == 1) & (risk_lkp['PERSON_TYPE'] == 'Organization') ]['RULE_NO'].to_list()
# non_stat_rules = tuple(non_stat_rules)

stat_rules = tuple(static_rules)

In [11]:
et_risk_df_per = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]", conn)

In [12]:
# Read risk_person_summary_stg_rules for the static rules
if len(non_stat_rules) == 1:
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('{}')".format(non_stat_rules[0]),conn)
    
elif(len(non_stat_rules)== 0):
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] with (nolock) where RULE_NO in ('')",conn)
    
else:
    non_stat_rules = tuple(non_stat_rules)
    et_risk_df_per_rules = pd.read_sql_query("SELECT * FROM [Fincore_db].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG_RULES] with (nolock) where RULE_NO in" + str (non_stat_rules) , conn)

# stat_rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_PERSON_SUMMARY_STG_RULES] where RULE_NO in "+str(tuple(static_rules)) , conn)


In [13]:
non_stat_rules

('O002003005',
 'O002003006',
 'O002001018',
 'O002001032',
 'O002001019',
 'O002001021',
 'O002001025',
 'O002001023',
 'O002001024',
 'O002001026',
 'O002001029',
 'O002003001',
 'O002003002',
 'O002003003',
 'O002003007',
 'O002003008',
 'O002001034',
 'O002001033',
 'O002001031',
 'O002001030',
 'O002001028',
 'O002001008',
 'O002001009',
 'O002001010',
 'O002001011',
 'O001005000',
 'O001006000',
 'O002001001',
 'O002001002',
 'O002001004',
 'O002001027',
 'O002001007')

In [14]:
et_risk_df_per_rules

ORGANIZATION_MASTER_ID     RULE_NO RULE_VALUE  RISK_SCORE_STATIC  \
0                         1  O001006000          7           10.00000   
1                         1  O002001001          5                nan   
2                         1  O002001002          2                nan   
3                         1  O002001004          6                nan   
4                         1  O002001007          7                nan   
..                      ...         ...        ...                ...   
356                      84  O002001008          1                nan   
357                      84  O002001009          1                nan   
358                      84  O002001019          2                nan   
359                      84  O002001021          1                nan   
360                      84  O002001028          1                nan   

    CREATE_DATE  
0    2023-02-27  
1    2023-02-27  
2    2023-02-27  
3    2023-02-27  
4    2023-02-27  
..          ...  
356  2023-02-27  
357  2023-02-27  
358  2023-02-27  
359  2023-02-27  
360  2023-02-27  

[361 rows x 5 columns]

In [15]:
auto_high_df = et_risk_df_per_rules[(et_risk_df_per_rules['RULE_NO'].isin(auto_high_rules)) & (et_risk_df_per_rules['RISK_SCORE_STATIC'] == 10)]
risk_per = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION]",conn)
auto_high_df = auto_high_df.groupby('ORGANIZATION_MASTER_ID')['RULE_NO'].unique().reset_index()
auto_high_df['RISK_RULE_LIST'] = auto_high_df['RULE_NO'].apply(lambda x:"_".join(x))
auto_high_df['RISK_SCORE'] = 10

In [16]:
auto_high_df

ORGANIZATION_MASTER_ID       RULE_NO RISK_RULE_LIST  RISK_SCORE
0                        1  [O001006000]     O001006000          10
1                        8  [O001006000]     O001006000          10
2                       25  [O001006000]     O001006000          10
3                       31  [O001006000]     O001006000          10
4                       32  [O001006000]     O001006000          10
5                       34  [O001006000]     O001006000          10
6                       37  [O001006000]     O001006000          10
7                       51  [O001006000]     O001006000          10
8                       55  [O001006000]     O001006000          10
9                       65  [O001006000]     O001006000          10
10                      67  [O001006000]     O001006000          10
11                      68  [O001006000]     O001006000          10
12                      69  [O001006000]     O001006000          10
13                      77  [O001006000]     O001006000          10

In [17]:
flaged_per_master_id = auto_high_df['ORGANIZATION_MASTER_ID'].to_list()
flaged_per_master_id

[1, 8, 25, 31, 32, 34, 37, 51, 55, 65, 67, 68, 69, 77]

In [18]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [19]:
summary_stg_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION_SUMMARY_STG]", conn)

In [22]:
temp = pd.merge(auto_high_df,summary_stg_df , on = 'ORGANIZATION_MASTER_ID',how = 'inner')

In [24]:
temp

ORGANIZATION_MASTER_ID       RULE_NO RISK_RULE_LIST  RISK_SCORE  \
0                        1  [O001006000]     O001006000          10   
1                        8  [O001006000]     O001006000          10   
2                       25  [O001006000]     O001006000          10   
3                       31  [O001006000]     O001006000          10   
4                       32  [O001006000]     O001006000          10   
5                       34  [O001006000]     O001006000          10   
6                       37  [O001006000]     O001006000          10   
7                       51  [O001006000]     O001006000          10   
8                       55  [O001006000]     O001006000          10   
9                       65  [O001006000]     O001006000          10   
10                      67  [O001006000]     O001006000          10   
11                      68  [O001006000]     O001006000          10   
12                      69  [O001006000]     O001006000          10   
13                      77  [O001006000]     O001006000          10   

       PEER_GROUP_ID         PAN            GSTIN      UNIQUE_COMPANY_ID  \
0   OTHERS_CHANGED_9  FDFCP5634K  04AABCU9603R1ZV  L01631KA2010PTC096845   
1   OTHERS_CHANGED_9  FDKCP5634K             None  L01632KA2025PTC096844   
2   OTHERS_CHANGED_0  ICCCC1231A  27AAACL1881P1Z3  U52395MH2012PTC064541   
3   OTHERS_CHANGED_4  GDFCP8425M             None                   None   
4   OTHERS_CHANGED_4  GDTCK8446M             None  L01631KA2050PTC764926   
5   OTHERS_CHANGED_9  ABCTY1234D             None  L17110MH1973PLC019786   
6   OTHERS_CHANGED_9  GDFCP3445K             None                   None   
7   OTHERS_CHANGED_2  NDTCK8416B             None  L09631KA2000PTC752010   
8   OTHERS_CHANGED_9  ABCTY1594D             None  L17110MH1973PLC029798   
9   OTHERS_CHANGED_0  FHJCP8123C  09AAACI8520G2Z3                   None   
10  OTHERS_CHANGED_7  NDTCK2110B             None  L09631KA2050PTC763001   
11  OTHERS_CHANGED_7  NDTCK8071B             None  L09631KA2150PTC767844   
12  OTHERS_CHANGED_9  NDTCK8787Z             None  L09631KA2050PTC876333   
13  OTHERS_CHANGED_9  HGFCP5871U  18AABCU9652R1ZM  L17110MH1953PLC019586   

           IEC                     FULL_NAME DATE_OF_INCORPORATION  \
0   FDFCP5633K                           TCS            2006-05-24   
1         None                       Infosys            2006-05-23   
2         None    Larsen and toubro Infotech                  None   
3         None                  Lnt Infotech            1974-01-25   
4         None                    Raj Motors            2020-06-09   
5         None                   Infosys ltd                  None   
6         None                   Lntmindtree            1975-09-12   
7         None       Rambhakt Power Industry            1951-09-26   
8         None                   Canara Bank                  None   
9         None        Singhaniya enterprises            2010-09-20   
10        None             Ishika Enterprize            1998-10-21   
11        None  Miva Scientific Technologies            1975-10-23   
12        None                Oisha Farmtech            1968-10-12   
13  0100000019          Adishakti Energy LTD            2010-09-08   

    CBDT_INCOME_RANGE   TRAN_AMNT_CREDIT   TRAN_AMNT_DEBIT    PEER_GROUP_AMNT  \
0                 nan    777781617.00000   152448826.00000    777781617.00000   
1                 nan 698150920003.00000               nan 698150920003.00000   
2                 nan                nan               nan            0.00000   
3                 nan                nan     2022750.00000      2022750.00000   
4                 nan                nan     2022750.00000      2022750.00000   
5                 nan   4469115312.00000 43479198218.00000   4469115312.00000   
6                 nan    574000000.00000               nan    574000000.00000   
7        750000.00000                nan    25000000.00000       750000.00000   
8                

In [25]:
# Adding the flaged person in auto high rules to risk person table

for ind,row in temp.iterrows():
    conn.execute('''INSERT INTO [FINCORE_DB].[Fincore_Analytics].[RISK_ORGANIZATION] (ORGANIZATION_MASTER_ID, PEER_GROUP_ID, RISK_SCORE)
    values (?,?,?)''',(row['ORGANIZATION_MASTER_ID'],row['PEER_GROUP_ID'],10))

conn.commit()
conn.close()


ProgrammingError: ('42S02', "[42S02] [FreeTDS][SQL Server]Invalid object name 'FINCORE_DB.Fincore_Analytics.ORGANIZATION_PERSON'. (208) (SQLExecDirectW)")

In [26]:
et_risk_df_per_rules = et_risk_df_per_rules[~et_risk_df_per_rules['ORGANIZATION_MASTER_ID'].isin(auto_high_df['ORGANIZATION_MASTER_ID'].to_list())]

In [27]:
et_risk_df_per_rules

ORGANIZATION_MASTER_ID     RULE_NO RULE_VALUE  RISK_SCORE_STATIC  \
10                        2  O002001007          1                nan   
11                        2  O002001008          1                nan   
12                        2  O002001025          1                nan   
13                        3  O002001007          1                nan   
14                        3  O002001008          1                nan   
..                      ...         ...        ...                ...   
356                      84  O002001008          1                nan   
357                      84  O002001009          1                nan   
358                      84  O002001019          2                nan   
359                      84  O002001021          1                nan   
360                      84  O002001028          1                nan   

    CREATE_DATE  
10   2023-02-27  
11   2023-02-27  
12   2023-02-27  
13   2023-02-27  
14   2023-02-27  
..          ...  
356  2023-02-27  
357  2023-02-27  
358  2023-02-27  
359  2023-02-27  
360  2023-02-27  

[266 rows x 5 columns]

In [28]:
et_risk_df_per_rules.drop_duplicates(inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
#changing vertical format to horizontal
pivot_per_df = et_risk_df_per_rules
pivot_per_df = pivot_per_df.pivot(index='ORGANIZATION_MASTER_ID', columns='RULE_NO', values=['RULE_VALUE'])
pivot_per_df.columns = [col[1] for col in pivot_per_df.columns.values]
pivot_per_df = pivot_per_df.reset_index()
cols = pivot_per_df.columns.to_list()
cols.append('CREATE_DATE')
pivot_per_df = pivot_per_df.merge(et_risk_df_per_rules,on = 'ORGANIZATION_MASTER_ID')[cols]


In [32]:
pivot_per_df.drop_duplicates(inplace = True)

In [34]:
data_ind_U = data_ind_U.merge(pivot_per_df,how = 'left',on = 'ORGANIZATION_MASTER_ID')
data_ind_U.head()

ORGANIZATION_MASTER_ID  PEER_GROUP_AMNT  PEER_GROUP_AMNT_Scaled  \
0                       1  777781617.00000                 0.00002   
1                       2          0.00000                 0.00000   
2                       3          0.00000                 0.00000   
3                       4    1500000.00000                 0.00000   
4                       5          0.00000                 0.00000   

      PEER_GROUP_ID  Cluster_labels OCCUPATION_BUSINESS_CLEAN O002001001  \
0  OTHERS_CHANGED_9               9            OTHERS_CHANGED        NaN   
1  OTHERS_CHANGED_0               0            OTHERS_CHANGED        NaN   
2  OTHERS_CHANGED_0               0            OTHERS_CHANGED        NaN   
3  OTHERS_CHANGED_3               3            OTHERS_CHANGED          1   
4  OTHERS_CHANGED_0               0            OTHERS_CHANGED        NaN   

  O002001002 O002001004 O002001007 O002001008 O002001009 O002001010  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN          1          1        NaN        NaN   
2        NaN        NaN          1          1        NaN        NaN   
3          1          1          1          1        NaN        NaN   
4        NaN        NaN          1          1        NaN        NaN   

  O002001011 O002001018 O002001019 O002001021 O002001023 O002001024  \
0        NaN        NaN        NaN        NaN        NaN        NaN   
1        NaN        NaN        NaN        NaN        NaN        NaN   
2        NaN        NaN        NaN        NaN        NaN        NaN   
3        NaN        NaN        NaN        NaN        NaN          1   
4        NaN        NaN        NaN        NaN        NaN          1   

  O002001025 O002001028 O002001034 O002003001 O002003002           O002003005  \
0        NaN        NaN        NaN        NaN        NaN                  NaN   
1          1        NaN        NaN        NaN        NaN                  NaN   
2          1        NaN        NaN        NaN        NaN                  NaN   
3        NaN        NaN          1    9508.58        NaN  -180.00000000000000   
4        NaN        NaN        NaN        NaN        NaN                  NaN   

          O002003006 CREATE_DATE  
0                NaN         NaN  
1                NaN  2023-02-27  
2                NaN  2023-02-27  
3  9558.582666666667  2023-02-27  
4                NaN  2023-02-27

In [110]:
data_ind_U = data_ind_U[~data_ind_U['ORGANIZATION_MASTER_ID'].isin(auto_high_df['ORGANIZATION_MASTER_ID'].to_list())]

In [111]:
data_ind_U.head()

ORGANIZATION_MASTER_ID  PEER_GROUP_AMNT  PEER_GROUP_AMNT_Scaled  \
0                       1  777781617.00000                 0.00002   
1                       2          0.00000                 0.00000   
2                       3          0.00000                 0.00000   
3                       4    1500000.00000                 0.00000   
4                       5          0.00000                 0.00000   

      PEER_GROUP_ID  Cluster_labels OCCUPATION_BUSINESS_CLEAN O002001001  \
0  OTHERS_CHANGED_9               9            OTHERS_CHANGED          5   
1  OTHERS_CHANGED_0               0            OTHERS_CHANGED        NaN   
2  OTHERS_CHANGED_0               0            OTHERS_CHANGED        NaN   
3  OTHERS_CHANGED_3               3            OTHERS_CHANGED          1   
4  OTHERS_CHANGED_0               0            OTHERS_CHANGED        NaN   

  O002001002 O002001004 O002001007 O002001008 O002001009 O002001010  \
0          2          6          7          6          1          1   
1        NaN        NaN          1          1        NaN        NaN   
2        NaN        NaN          1          1        NaN        NaN   
3          1          1          1          1        NaN        NaN   
4        NaN        NaN          1          1        NaN        NaN   

  O002001011 O002001018 O002001021 O002001023 O002001024 O002001025  \
0        NaN        NaN        NaN        NaN          9          9   
1        NaN        NaN        NaN        NaN          1          1   
2        NaN        NaN        NaN        NaN          1          1   
3        NaN        NaN        NaN        NaN          2        NaN   
4        NaN        NaN        NaN        NaN          1        NaN   

  O002001027 O002001028 O002001030 O002001031 O002001033 O002001034  \
0          9        NaN          9        NaN          9        NaN   
1          1        NaN          1        NaN          1        NaN   
2          1        NaN          1        NaN          1        NaN   
3          2        NaN          2        NaN          2          1   
4          1        NaN          1        NaN          1        NaN   

  O002003001 O002003002             O002003005            O002003006  \
0        NaN        NaN  128481.07000000000000  1594783.706000000000   
1        NaN        NaN                    NaN                   NaN   
2        NaN        NaN                    NaN                   NaN   
3    9508.58        NaN    -180.00000000000000     9558.582666666667   
4        NaN        NaN                    NaN                   NaN   

  O002003008 CREATE_DATE  
0        NaN  2023-02-23  
1        NaN  2023-02-23  
2        NaN  2023-02-23  
3        NaN  2023-02-23  
4        NaN  2023-02-23

In [114]:
reqd_cols_per = list(set(non_stat_rules) - set(auto_high_rules)) #doing this to remove auto high rule from nan_stat_rule_list

# Threshold Calculations without Outliers

In [121]:
def compute_IQR(d,data):
    print("compute_IQR")
    q3 = np.quantile(data, 0.75)
    q1 = np.quantile(data, 0.25)
    iqr = ((q3-q1)* 1.5)
    outliers_left = q1 - iqr
    outliers_right = q3 + iqr
    print("outliers_right",outliers_right)
#     data = list(filter(lambda x: outliers_left <= x <= outliers_right, data))
    data = list(filter(lambda x: x <= outliers_right,d))

    return data,outliers_right

In [122]:
def normal_outliers_removal(d):
    print("normal_outliers_removal")
    mean = np.mean(d)
    std = np.std(d)
    outliers_right = (mean + (3*std))
    data = list(filter(lambda x: x <= outliers_right, d))
    return data,outliers_right
    
    

In [123]:
def thresh_cal(d,outliers_right):
    print("thresh")
    median = np.median(d)
    mean = np.mean(d)
    std = np.std(d)
    mode = int(s.mode(d)[0])
    print("DATA = ",d,median,std,"********************************")
    
    if std >= 1:
        one = round(median+std)
        two = round(median+2*std)
        three = round(median+3*std)
        four = round(median+4*std)
        five = round(median+5*std)
        six = round(median+6*std)
        seven = round(median+7*std)
        eight = round(median+8*std)
        nine =round(median+9*std)
        ten = round(median+10*std)

    elif std > 0:
        one = np.ceil(median+std)
        two = np.ceil(one+std)
        three = np.ceil(two+std)
        four = np.ceil(three+std)
        five = np.ceil(four+std)
        six = np.ceil(five+std)
        seven = np.ceil(six+std)
        eight = np.ceil(seven+std)
        nine =np.ceil(eight+std)
        ten = np.ceil(nine+std)
        
        
    elif std == 0:
        print("STD = 0")
        one = np.ceil(median+std+1)
        two = np.ceil(one+std+1)
        three = np.ceil(two+std+1)
        four = np.ceil(three+std+1)
        five = np.ceil(four+std+1)
        six = np.ceil(five+std+1)
        seven = np.ceil(six+std+1)
        eight = np.ceil(seven+std+1)
        nine =np.ceil(eight+std+1)
        ten = np.ceil(nine+std+1)

        

    return one,two,three,four,five,six,seven,eight,nine,ten,mean,median,mode,std,outliers_right

In [124]:
def not_normal_outliers_removal(d):
    print("not_normal_outliers_removal")
    print(d)
    data = [*set(d)]
    return compute_IQR(d,data)

In [125]:
def compute_outlier(data,col):
    try:
#         print("data = ",data)
        d = data[col].dropna().astype('float64')
        d = d.astype('int64')
#         print("d = ",d)
    
        if (len(d) == 0):
            print("Not applicable")
            return None,None,None,None,None,None,None,None,None,None
          

        elif ( len(d) < 5):
            print("Not applicable")
            return thresh_cal(d,"NA") 
        
        else:
            
            if ( len(d) <= 150):
                stats,p_val = shapiro(d)
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)   
                    
            else:
                stats,p_val  = kstest(d, 'norm')
                if p_val > 0.05:
                    d,outliers_right = normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
                    
                else:
                    d,outliers_right = not_normal_outliers_removal(d)
                    return thresh_cal(d,outliers_right)
    
            
    except KeyError:
        return( None,None,None,None,None,None,None,None,None,None) 
    
    return (None,None,None,None,None,None,None,None,None,None)

In [126]:
class Thresholds():
    def __init__(self,df,reqd_cols):
        self.data = df
        self.reqd_cols_U = reqd_cols
    def get_thresholds(self):
        # Python3 dict to contain threshold calculations-
        # Python3 lists to contain threshold values for desired columns-       
        self.class_dict = {}
        for col in self.reqd_cols_U:
            self.class_dict[col] = []
        
        self.class_dict['OCCUPATION_BUSINESS_CLEAN'] = []
        self.class_dict['Cluster_labels'] = []

        for occ in self.data['OCCUPATION_BUSINESS_CLEAN'].unique():
#             if occ == 'Others':
                for clabel in self.data.loc[self.data['OCCUPATION_BUSINESS_CLEAN'] == occ, 'Cluster_labels'].unique():

                    for col in self.reqd_cols_U:
                        print(f"Profession: {occ}, Cluster = {clabel},"
                        f" col = {col}; outlier"
                        f" = {compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col)}"
                        )
                        # profession.append(occ)
                        # profession.append(clabel)
                        self.class_dict[col].append(compute_outlier(self.data[(self.data['OCCUPATION_BUSINESS_CLEAN'] == occ) & (self.data['Cluster_labels'] == clabel)], col))

                    self.class_dict['OCCUPATION_BUSINESS_CLEAN'].append(occ)
                    self.class_dict['Cluster_labels'].append(clabel)
#                 print(self.class_dict)
#                 break
#         self.class_dict['Type_of_profession'] = []
#         self.class_dict['Cluster_labels'] = []
                        
#         for occ in self.data['Type_of_profession'].unique():
#             for clabel in self.data.loc[self.data['Type_of_profession'] == occ, 'Cluster_labels'].unique():
#                 # print(f"{occ}, {clabel}")
#                 self.class_dict['Type_of_profession'].append(occ)
#                 self.class_dict['Cluster_labels'].append(clabel)
        
        self.threshold_data = pd.DataFrame(self.class_dict)
        return self.threshold_data
#         return self.class_dict

In [127]:
data_ind_U.drop_duplicates(inplace = True)

In [128]:
ind_U = Thresholds(data_ind_U,reqd_cols_per)
ind_data_thresholds_U = ind_U.get_thresholds()

Not applicable
thresh
DATA =  14    1
Name: O002001011, dtype: int64 1.0 0.0 ********************************
STD = 0
Profession: OTHERS_CHANGED, Cluster = 9, col = O002001011; outlier = (2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 1.0, 1.0, 1, 0.0, 'NA')
Not applicable
thresh
DATA =  14    1
Name: O002001011, dtype: int64 1.0 0.0 ********************************
STD = 0
not_normal_outliers_removal
40    1
41    2
53    2
56    2
57    1
58    1
61    2
66    1
67    1
Name: O002001028, dtype: int64
compute_IQR
outliers_right 2.5
thresh
DATA =  [1, 2, 2, 2, 1, 1, 2, 1, 1] 1.0 0.4969039949999533 ********************************
Profession: OTHERS_CHANGED, Cluster = 9, col = O002001028; outlier = (1, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 1.4444444444444444, 1.0, 1, 0.4969039949999533, 2.5)
not_normal_outliers_removal
40    1
41    2
53    2
56    2
57    1
58    1
61    2
66    1
67    1
Name: O002001028, dtype: int64
compute_IQR
outliers_right 2.5
thresh
DATA =  [1, 2, 2, 

/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
/opt/conda/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "



not_normal_outliers_removal
1     1
2     1
4     1
8     1
9     1
10    1
11    1
12    1
16    1
17    1
22    1
24    4
28    2
30    1
33    1
34    1
35    1
36    1
37    1
49    1
62    1
63    1
64    1
Name: O002001007, dtype: int64
compute_IQR
outliers_right 5.25
thresh
DATA =  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1.0 0.6360321234055564 ********************************
Profession: OTHERS_CHANGED, Cluster = 0, col = O002001007; outlier = (2, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 1.173913043478261, 1.0, 1, 0.6360321234055564, 5.25)
not_normal_outliers_removal
1     1
2     1
4     1
8     1
9     1
10    1
11    1
12    1
16    1
17    1
22    1
24    4
28    2
30    1
33    1
34    1
35    1
36    1
37    1
49    1
62    1
63    1
64    1
Name: O002001007, dtype: int64
compute_IQR
outliers_right 5.25
thresh
DATA =  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] 1.0 0.6360321234055564 ***************************

In [131]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [133]:
weight_sql = pd.read_sql_query("SELECT * FROM [Fincore_DB].[Fincore_Analytics].[RISK_LKP_RULES]", conn)

In [134]:
weight_table_ind_U = ind_data_thresholds_U.copy()

In [135]:
for col in ind_data_thresholds_U.columns:
    if (col == 'OCCUPATION_BUSINESS_CLEAN') or (col == 'Cluster_labels') or (col == 'TRAN_AMNT'):
        pass
    else:
        weight_table_ind_U[col] = weight_sql[weight_sql['RULE_NO'] == col]['WEIGHTAGE'].values[0]

In [139]:
config = ConfigParser()
config.read(r'/notebooks/notebooks/config.ini')
connection_string = config.get('SERVER', 'connection')
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [140]:
rule_df = pd.read_sql_query("SELECT * FROM [FINCORE_DB].[Fincore_Analytics].[RISK_LKP_RULES] where IS_ACTIVE_FLAG = 1 and RISK_SCORE_STATIC_FLAG = 0 and RISK_TYPE <> 'Auto High Rules' and PERSON_TYPE = 'Organization'", conn)
reqd_cols_per = rule_df['RULE_NO'].to_list()
len(reqd_cols_per)

30

In [141]:
conn.close()

In [142]:
col_with_data = data_ind_U.columns.to_list()

In [143]:
col_with_data = [col for col in col_with_data if 'O0' in col]

In [146]:
col_without_data = set(reqd_cols_per) - set(col_with_data)

In [148]:
for col in col_without_data:
    data_ind_U[col] = np.nan

In [151]:
ind_data_thresholds_U.to_csv(r"/data/threshold_individual_U.csv",index=False)
data_ind_U.to_csv(r"/data/Synthetic_data_Clustered_individual_U.csv",index=False)
weight_table_ind_U.to_csv(r"/data/weight_individual_U.csv",index=False)

In [1]:
print('Threshold_Computation_V2_ORG completed.')

Threshold_Computation_V2_ORG completed.
